#### Home Website Scarping

First we need to know what social media each company have.

By using the website link we scraped from YAHOO! Finance, we go to each website to see what social media it has and what's the link.

In [ ]:
import numpy as np
import pandas as pd
import requests
import bs4
import re
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# read the list of public companies information scrapped from YAHOO! Finance
df = pd.read_csv('listing.csv', index_col="'Unnamed: 0'")
df['facebook'] = None
df['twitter'] = None
df['linkedin'] = None
df['youtube'] = None
df['instagram'] = None

In [ ]:
# scrap using request to get first version
for i in df.index:
    url = df['website'].loc[i]
    print((i, url))
    agent = {"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.87 Safari/537.36'}
    cookies = {"cookie":"COPY_HERE_YOUR_COOKIE_FROM_BROWSER"}
    try:
        html = requests.get(url, headers=agent, cookies=cookies)
        response = requests.get(html.url, headers=agent, cookies=cookies)
        if html.status_code == 200:
            soup = bs4.BeautifulSoup(html.content,"html.parser")
            try:
                facebook = soup.find(href=re.compile('facebook')).get('href')
                df['facebook'].loc[i] = facebook
            except:
                print('no facebook')
            try:
                twitter = soup.find(href=re.compile('twitter')).get('href')
                df['twitter'].loc[i] = twitter
            except:
                print('no twitter')
            try:
                linkedin = soup.find(href=re.compile('linkedin')).get('href')
                df['linkedin'].loc[i] = linkedin
            except:
                print('no linkedin')
            try:
                instagram = soup.find(href=re.compile('instagram')).get('href')
                df['instagram'].loc[i] = instagram
            except:
                print('no instagram')
            try:
                youtube = soup.find(href=re.compile('youtube')).get('href')
                df['youtube'].loc[i] = youtube
            except:
                print('no youtube')
        else:
            print('forbidden')
    except:
        print('request failed')
# df.to_csv(r'/Users/darren/Desktop/media.csv', index = False)

In [ ]:
# there are many websites that request can't get all the information, so use selenium to update the information
media = pd.read_csv("update.csv")
# get the data that doesn't scrap from 1st version
nomedia = media[media['facebook'].isnull() & media['twitter'].isnull() & media['linkedin'].isnull() &
     media['youtube'].isnull() & media['instagram'].isnull()]
# set the driver
driver = webdriver.Chrome(ChromeDriverManager().install())
# scrap using selenium to get the second version
count = 1
for i in nomedia.index:
    url = nomedia['website'].loc[i]
    agent = {"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.87 Safari/537.36'}
    cookies = {"cookie":"COPY_HERE_YOUR_COOKIE_FROM_BROWSER"}
    print(count, url)
    try:
        html = requests.get(url, headers=agent, cookies=cookies, verify=False)
        driver.get(html.url)
        try:
            facebook = driver.find_element_by_xpath("*//a[contains(@href,'facebook')]").get_attribute('href')
            nomedia['facebook'].loc[i] = facebook
        except:
            print('no facebook')
        try:
            twitter = driver.find_element_by_xpath("*//a[contains(@href,'twitter')]").get_attribute('href')
            nomedia['twitter'].loc[i] = twitter
        except:
            print('no twitter')
        try:
            linkedin = driver.find_element_by_xpath("*//a[contains(@href,'linkedin')]").get_attribute('href')
            nomedia['linkedin'].loc[i] = linkedin
        except:
            print('no linkedin')
        try:
            youtube = driver.find_element_by_xpath("*//a[contains(@href,'youtube')]").get_attribute('href')
            nomedia['youtube'].loc[i] = youtube
        except:
            print('no youtube')
        try:
            instagram = driver.find_element_by_xpath("*//a[contains(@href,'instagram')]").get_attribute('href')
            nomedia['instagram'].loc[i] = instagram
        except:
            print('no instagram')
    except:
        print('request failed')
    count += 1
new = nomedia[~(nomedia['facebook'].isnull() & nomedia['twitter'].isnull() & nomedia['linkedin'].isnull() &
     nomedia['youtube'].isnull() & nomedia['instagram'].isnull())]
# update with 2nd version
for i in new.index:
    media['facebook'][i] = new['facebook'][i]
    media['twitter'][i] = new['twitter'][i]
    media['linkedin'][i] = new['linkedin'][i]
    media['youtube'][i] = new['youtube'][i]
    media['instagram'][i] = new['instagram'][i]
# show the new scraping results
media[media['facebook'].isnull() & media['twitter'].isnull() & media['linkedin'].isnull() &
     media['youtube'].isnull() & media['instagram'].isnull()]  
# media.to_csv(r'/Users/darren/Desktop/latest.csv', index = False)
    